In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import cleverhans
from cleverhans.attacks import FastGradientMethod, MadryEtAl, MomentumIterativeMethod
from keras.applications.resnet50 import ResNet50
from keras.applications.densenet import DenseNet121
from keras.applications.xception import Xception
from keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg16 import VGG16
from cleverhans.utils_keras import KerasModelWrapper
from keras.applications.imagenet_utils import preprocess_input
from cleverhans.model import EnsembleWrapper
from keras.layers.core import Lambda
from keras.layers import Input
import numpy as np
import tensorflow as tf
from keras.layers import Input
from keras.models import Model
from keras.layers import Average
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [ ]:
alpha = 3.0
clip_min = 0.0
clip_max = 1.0

config = tf.ConfigProto()
config.gpu_options.allow_growth=True

with tf.Session(config=config) as sess:
    input_tensor = Input(shape=(224, 224, 3))
    tf_inputs = Lambda(lambda x: preprocess_input(x, mode='tf'))(input_tensor)
    caffe_inputs = Lambda(lambda x: preprocess_input(x, mode='caffe'))(input_tensor)

    base_inception = InceptionV3(input_tensor=input_tensor, weights="imagenet", include_top=True)
    inception = Model(inputs=input_tensor, outputs=base_inception(tf_inputs))

    base_densenet = DenseNet121(input_tensor=input_tensor, weights="imagenet", include_top=True)
    densenet = Model(inputs=input_tensor, outputs=base_densenet(tf_inputs))

#     base_resnet = ResNet50(input_tensor=input_tensor, weights="imagenet", include_top=True)
#     resnet = Model(inputs=input_tensor, outputs=base_resnet(caffe_inputs))

#     base_vgg = VGG16(input_tensor=input_tensor, weights="imagenet", include_top=True)
#     vgg = Model(inputs=input_tensor, outputs=base_vgg(caffe_inputs))

#     base_xception = Xception(input_tensor=input_tensor, weights="imagenet", include_top=True)
#     xception = Model(inputs=input_tensor, outputs=base_xception(tf_inputs))
    
#     models = [inception, xception, resnet, densenet, vgg]
    models = [inception, densenet]

    for model in models:
        model.compile(optimizer="adam", loss='categorical_crossentropy', metrics=['accuracy'])
    
    X_exp = np.load('dl_experiments_data/imagenet/X_exp.npy')[:100]
    Y_exp = np.load('dl_experiments_data/imagenet/Y_exp.npy')[:100]
    
    models = [EnsembleWrapper(model) for model in models]
    
    model_attacks = []
    
    for model in models:
        
        fgm = FastGradientMethod(model, sess=sess)
        madry = MadryEtAl(model, sess=sess)
        mim = MomentumIterativeMethod(model, sess=sess)

        fgm_params = {'eps': alpha,
                      'clip_min': clip_min,
                      'clip_max': clip_max,
                      'ord': 2,
                      'y': Y_exp}

        adv_fgm = fgm.generate_np(X_exp, **fgm_params)

        madry_params = {'eps': alpha,
                        'eps_iter': .01,
                        'nb_iter': 400,
                        'clip_min': clip_min,
                        'clip_max': clip_max, 
                        'ord': 2,
                        'y': Y_exp}

        adv_madry = madry.generate_np(X_exp, **madry_params)

        mim_params = {'eps': alpha,
                      'eps_iter': alpha / 5.0,
                      'nb_iter': 5,
                      'clip_min': clip_min,
                      'clip_max': clip_max, 
                      'ord': 2, 
                      'decay_factor': 1.0,
                      'y': Y_exp}

        adv_mim = mim.generate_np(X_exp, **mim_params)
        
        model_attacks.append([adv_fgm, adv_madry, adv_mim])

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

